In [1]:
import pycellin as pc

### Adding pre-defined features

In [2]:
xml_path = "../sample_data/Ecoli_growth_on_agar_pad.xml"
model = pc.load_TrackMate_XML(xml_path)
print(model)

Model with 3 lineages.


In [3]:
pc.get_pycellin_cell_lineage_features()
model.add_pycellin_features(["relative_age", "cell_length"])
model.add_cell_width(
    method_width="median", width_ignore_tips=True, rename="cell_width_no_tips"
)
model.add_cycle_data()



model.add_pycellin_features(["division_time", "cell_cycle_completeness"])
model.update()

In [4]:
lin0 = model.data.cell_data[0]
lin0.plot(
    node_hover_features=["cell_ID", "relative_age", "cell_length", "cell_width_no_tips"]
)

In [5]:
cyclelin0 = model.data.cycle_data[0]
cyclelin0.plot(
    title="Cycle lineage of lineage ID 0",
    node_hover_features=[
        "cycle_ID",
        "level",
        "division_time",
        "cell_cycle_completeness",
    ],
)

### Adding custom feature

In [6]:
feat = pc.Feature(
    name="node_ID_parity",
    description="Parity of the node ID",
    lineage_type="CellLineage",
    provenance="Pycellin",
    data_type="int",
)


class ParityCalculator(pc.NodeLocalFeatureCalculator):

    def compute(self, lineage, noi):
        # `lineage` is unused here, but it is still a required argument.
        if noi % 2 == 0:
            return True
        else:
            return False

In [7]:
model.add_custom_feature(feat, ParityCalculator)
model.update()

### Changing the lineages structures and updating the features

In [8]:
new_cell_ID = model.add_cell(lineage_ID=0, frame=21)
model.link_cells(source_cell_ID=9326, source_lineage_ID=0, target_cell_ID=new_cell_ID)

In [9]:
model.is_update_required()

True

In [10]:
# We remove morphology related features since we don't have the data for them for the new cell.
model.remove_feature("cell_length", "node")
model.remove_feature("cell_width_no_tips", "node")
model.update()

In [11]:
lin0 = model.data.cell_data[0]
lin0.plot(node_hover_features=["cell_ID", "relative_age", "node_ID_parity"])

### Node age

In [1]:
import pycellin as pc

In [2]:
# Path to the TrackMate XML file.
xml_path1 = "../sample_data/Ecoli_growth_on_agar_pad.xml"
xml_path2 = "../sample_data/Ecoli_growth_on_agar_pad_with_gaps.xml"


# Parse the XML file and create a Pycellin Model object
# that contains all the data from the XML file.
model1 = pc.load_TrackMate_XML(xml_path1)
model2 = pc.load_TrackMate_XML(xml_path2)

# We can display basic information about this model.
print(model1)
print(f"This model contains {model1.data.number_of_lineages()} lineages:")
for lin_ID, lineage in model1.data.cell_data.items():
    print(f"- ID {lin_ID}: {lineage}")

Model with 3 lineages.
This model contains 3 lineages:
- ID 0: CellLineage named 'Track_0' with 152 nodes and 151 edges
- ID 1: CellLineage named 'Track_1' with 189 nodes and 188 edges
- ID 2: CellLineage named 'Track_2' with 185 nodes and 184 edges


In [3]:
model1.add_pycellin_features(["absolute_age", "relative_age"])
model1.update()

In [4]:
lin0_1 = model1.data.cell_data[0]

In [5]:
lin0_1.plot(node_hover_features=["cell_ID", "absolute_age", "relative_age"])

In [6]:
model2.add_pycellin_features(["absolute_age", "relative_age"])
model2.update()
lin0_2 = model2.data.cell_data[0]
lin0_2.plot(node_hover_features=["cell_ID", "absolute_age", "relative_age"])

### No segmentation

In [11]:
import pycellin as pc

In [12]:
xml = "E:/Pasteur/LS_data/LStoLX/230328GreffeGakaYFPMyogTdtmdxFDBTryplen1-movie01-01-Scene-15-TR37-A01.xml"
model = pc.load_TrackMate_XML(xml)
print(model)

Model with 1 lineage.


In [13]:
model.add_pycellin_features(["relative_age", "absolute_age"])
model.add_cycle_data()
model.add_pycellin_features(["division_time", "cell_cycle_completeness"])
model.update()

In [14]:
model.get_cycle_lineage_features()

['cycle_ID',
 'cells',
 'cycle_length',
 'level',
 'division_time',
 'cell_cycle_completeness',
 'cycle_lineage_ID']

In [15]:
lin0 = model.data.cell_data[0]
lin0.plot(node_hover_features=["cell_ID", "relative_age", "absolute_age"])

In [16]:
cyclelin0 = model.data.cycle_data[0]
cyclelin0.plot(
    title="Cycle lineage of lineage ID 0",
    node_hover_features=[
        "cycle_ID",
        "level",
        "division_time",
        "cell_cycle_completeness",
    ],
)

### EpiCure

In [1]:
import pickle

import napari

In [2]:
epi_file = "/mnt/data/Code/EpiCure_small_example/epics/013_crop_epidata.pkl"

In [4]:
with open(epi_file, "rb") as f:
    epidata = pickle.load(f)

for k, v in epidata.items():
    print(k, v)

EpiMetaData {'ScaleXY': 1, 'UnitXY': 'um', 'ScaleT': 1, 'UnitT': 'min', 'MainChannel': 0, 'Allow gaps': True, 'Verbose': 2}
Group {'paf': [48, 60, 70]}
Graph {167: [24], 172: [24], 393: [112], 397: [112], 492: [76], 505: [76], 645: [505], 658: [505], 1013: [33], 1016: [33], 1059: [916], 1066: [916], 1204: [82], 1200: [82], 1237: [674], 1240: [674], 1309: [46], 1321: [46], 1522: [1380], 1526: [1380], 208: [65, 72], 306: [167, 172], 674: [393, 397], 916: [492, 645], 1293: [32, 1016], 1345: [1200, 1204], 1380: [1237, 1240], 1465: [1321, 58]}
Events {'Points': array([[  1,  22,  30],
       [  2, 139,  68],
       [  3,  99,  19],
       [  4, 111,  15],
       [  7,  31, 212],
       [  7,  99,  19],
       [  8,  93,  43],
       [  8, 138,  68],
       [  9,  53, 197],
       [ 10, 138,  69]]), 'Props': {'id': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint16), 'label': array([  24,  112,   76,  505,   33,  916,   82,  674,   46, 1380],
      dtype=uint16), 'score': array([1, 1, 1, 1, 

In [6]:
graph = epidata["Graph"]
print(graph)

{167: [24], 172: [24], 393: [112], 397: [112], 492: [76], 505: [76], 645: [505], 658: [505], 1013: [33], 1016: [33], 1059: [916], 1066: [916], 1204: [82], 1200: [82], 1237: [674], 1240: [674], 1309: [46], 1321: [46], 1522: [1380], 1526: [1380], 208: [65, 72], 306: [167, 172], 674: [393, 397], 916: [492, 645], 1293: [32, 1016], 1345: [1200, 1204], 1380: [1237, 1240], 1465: [1321, 58]}
